### Computational baseline test

Approximate average elapsed time for n = 50000000:
- 1-node cluster: 8.0 (+/- 0.1) seconds
- 2-node cluster: 7.0 (+/- 0.1) seconds
- 3-node cluster: 7.4 (+/- 0.1) seconds

In [ ]:
import time

n = 50000000
count = 0
start_time = time.monotonic()

for i in range(n):
    count += 1

print('execution time: ', time.monotonic() - start_time)

### TensorFlow (Keras) MNIST ANN test

https://www.tensorflow.org/datasets/keras_example

Approximate average elapsed time for epochs = 6:
- 1-node cluster: 51.0 (+/- 2.0) seconds
- 2-node cluster: 41.0 (+/- 2.0) seconds
- 3-node cluster: 35.0 (+/- 2.5) seconds

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import time

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [ ]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [ ]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [ ]:
start_time = time.monotonic()

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

print('\nexecution time: ', time.monotonic() - start_time)